In [ ]:
# 태양고도 등 데이터 수집을 위한 크롤링 방법

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# 칼럼 크롤링

In [7]:
html = urlopen("https://astro.kasi.re.kr/life/pageView/10?useElevation=1&lat=36.813805376962705&lng=127.05378294561001&elevation=-118.3559296513817&output_range=1&date=2019-01-01&hour=&minute=&second=&address=%EC%B6%A9%EC%B2%AD%EB%82%A8%EB%8F%84+%EC%95%84%EC%82%B0%EC%8B%9C+%ED%83%95%EC%A0%95%EB%A9%B4+%ED%83%95%EC%A0%95%EB%A1%9C+212")

bsObject = BeautifulSoup(html, "html.parser")

target = bsObject.find('div', {'class':'table-responsive','id':'sun-height-table'})
target

<div class="table-responsive" id="sun-height-table">
<table class="table tbl_cnt tbl_border tbl_center">
<caption>태양 고도/방위각 계산의 검색결과에 따른 시간, 방위각, 고도, 적경, 적위를 확인하실 수 있습니다.</caption>
<colgroup>
<col class="multi-record-only" style="width:20%;"/>
<col style="width:20%;"/>
<col style="width:20%;"/>
<col style="width:20%;"/>
<col style="width:20%;"/>
</colgroup>
<thead>
<tr>
<th class="multi-record-only" id="date-col" scope="col">시간(시)</th>
<th scope="col">방위각(도 분 초)</th>
<th scope="col">고도(도 분 초)</th>
<th scope="col">적경(시 분 초)</th>
<th scope="col">적위(도 분 초)</th>
</tr>
</thead>
<tbody>
</tbody>
</table>
</div>

In [8]:
thead = target.find_all('th')

theadList = []

theadLen = len(thead)
for i in range(0,theadLen):
    thead = target.find_all('th')[i].text
    theadList.append(thead)
    
theadList

['시간(시)', '방위각(도 분 초)', '고도(도 분 초)', '적경(시 분 초)', '적위(도 분 초)']

# 동적 데이터 크롤링
## 위 사이트의 데이터는 동적 크롤링을 사용해야 불러올 수 있다

In [ ]:
# 크롬드라이버 이용시 크롬버전과 크롬드라이버 버전이 같아야 실행시킬 수 있습니다.

In [14]:
from selenium import webdriver
import datetime
driver = webdriver.Chrome('C:/Users/ASUS/Downloads/chromedriver_win32/chromedriver')
    
driver.implicitly_wait(1)
for date in pd.date_range(start = "2019-01-01", end = '2019-01-01'):
    k = date.strftime("%Y-%m-%d")
    url = f"https://astro.kasi.re.kr/life/pageView/10?useElevation=1&lat=36.813805376962705&lng=127.05378294561001&elevation=-118.3559296513817&output_range=1&date={k}&hour=&minute=&second=&address=%EC%B6%A9%EC%B2%AD%EB%82%A8%EB%8F%84+%EC%95%84%EC%82%B0%EC%8B%9C+%ED%83%95%EC%A0%95%EB%A9%B4+%ED%83%95%EC%A0%95%EB%A1%9C+212"
    driver.get(url)
    driver.implicitly_wait(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    table=soup.find("table",{"class":"table tbl_cnt tbl_border tbl_center"})
    td = table.find_all("td")
    

    hour=[]
    one = []
    altitude=[]
    two = []
    three=[]

    for i in range(0,len(td)):
        if i%5==0:
            hour.append(td[i].text)
        elif i%5==1:
            one.append(td[i].text)
        elif i%5==2:
            altitude.append(td[i].text)
        elif i%5==3:
            two.append(td[i].text)
        elif i%5==4:
            three.append(td[i].text)
            
    df = pd.DataFrame(index=range(0,24), columns=theadList)
    df["시간(시)"] = hour
    df["방위각(도 분 초)"] = one
    df["고도(도 분 초)"] = altitude
    df["적경(시 분 초)"] = two
    df["적위(도 분 초)"] = three
    df["날짜"] = k
        #df.to_csv(f"C:/Users/ASUS/Desktop/탕정면 태양고도/{k}.csv")
        #driver.close()

In [15]:
df

,시간(시),방위각(도 분 초),고도(도 분 초),적경(시 분 초),적위(도 분 초),날짜
0,00,328 57 27.26,-74 20 29.04,18 42 58.09,-23 04 08.90,2019-01-01
1,01,023 17 14.30,-75 13 11.76,18 43 09.27,-23 03 57.53,2019-01-01
2,02,059 04 43.86,-67 05 03.40,18 43 20.45,-23 03 46.25,2019-01-01
3,03,076 29 10.63,-55 56 42.83,18 43 31.62,-23 03 35.06,2019-01-01
4,04,087 29 55.65,-44 03 56.84,18 43 42.77,-23 03 23.93,2019-01-01
5,05,096 12 30.55,-32 04 41.50,18 43 53.89,-23 03 12.85,2019-01-01
6,06,104 10 36.43,-20 16 17.98,18 44 04.97,-23 03 01.77,2019-01-01
7,07,112 12 55.00,-8 52 21.65,18 44 16.02,-23 02 50.67,2019-01-01
8,08,120 54 05.87,01 51 51.34,18 44 27.03,-23 02 39.50,2019-01-01
9,09,130 44 58.66,11 36 09.40,18 44 38.01,-23 02 28.23,2019-01-01
